## In this notebook, we scale the nll with the mean of the nll of the previous epoch and scale the cosine similaity with (1-lambda)

In [1]:
#installing some libraries
!pip install datasets
!pip install --upgrade jupyter ipywidgets



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch, transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import PreTrainedModel, PretrainedConfig
from transformers import AutoModel, AutoConfig,AutoModelForCausalLM, AutoConfig


import torch.nn as nn
import torch.nn.functional as F
#from datasets import load_dataset
import pandas as pd, numpy as np
from torch import cuda
import datetime
import warnings,itertools
from torch.optim.lr_scheduler import LambdaLR
from torch.cuda.amp import autocast, GradScaler
import json,math
pre_train = False

# Ignore all warnings
warnings.filterwarnings('ignore')
#pip install transformers bitsandbytes>=0.39.0 -q
import zipfile,logging

In [3]:
#global params for training

B,T = 32,1024
epoch = 100
# this controls whether we are using pre-trained wts or not
random_init_wts = False
if cuda.is_available():
    device = torch.device('cuda:0')
    print(device)
else:
    device = 'cpu'
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
# these 2 global vars help track the training and val loss
global_tr_loss = torch.inf
global_val_loss = torch.inf
#directory to save wts
model_path = os.path.join("model")


cuda:0


In [4]:
directory = os.path.join('.','data','unzip_text_10M')  # Replace with your directory path
directory

'./data/unzip_text_10M'

In [5]:
def read_text(directory):
    """This function loads the dataset,provided in the zipped format and stores all the text files in list
    each file has its contents stored as an item in list and at the end we concatenate all the sub-lists to create a flattened list and return it"""
    directory = os.path.join('.','data','unzip_text_10M',str(directory))  # Replace with your directory path
    print(f"directory :{directory}")
    # List all files in the directory
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    print(f"files:{files}")
    text_content = []
    # Read each file
    total_lines = 0
    for filenum,filename in enumerate(files):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            text_content.append(text)
            print(f"the file:{filename} has been appeneded to the uber list and its length is {len(text_content)} ")
            
    
    flattened_list = ''.join(text_content)
    assert (len(flattened_list) == total_lines , f"Expected {len(flattened_list)} to be equal to {total_lines}" )
    
    return flattened_list

    

In [6]:
#read the dataset and get a list
train_list = read_text("train_10M")


directory :./data/unzip_text_10M/train_10M
files:['switchboard.train', 'simple_wiki.train', 'open_subtitles.train', 'gutenberg.train', 'childes.train', 'bnc_spoken.train']
the file:switchboard.train has been appeneded to the uber list and its length is 1 
the file:simple_wiki.train has been appeneded to the uber list and its length is 2 
the file:open_subtitles.train has been appeneded to the uber list and its length is 3 
the file:gutenberg.train has been appeneded to the uber list and its length is 4 
the file:childes.train has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.train has been appeneded to the uber list and its length is 6 


In [7]:
len(train_list)

54215049

In [8]:
chunks = len(train_list)//(B*T)
print(chunks)

1654


In [9]:
#tokeinze the training data
tokenizer = AutoTokenizer.from_pretrained("gpt2",return_tensors = "pt" , truncate = True, return_overflowing_tokens=True , padding = False,)
enc_train = tokenizer(train_list)


Token indices sequence length is longer than the specified maximum sequence length for this model (17191971 > 1024). Running this sequence through the model will result in indexing errors


In [10]:
comp_ratio = len(train_list)/len(enc_train['input_ids'])
comp_ratio

3.1535097982657136

In [11]:
#read validation data and tokenize
val_list = read_text("dev")
enc_val = tokenizer(val_list)

directory :./data/unzip_text_10M/dev
files:['switchboard.dev', 'simple_wiki.dev', 'open_subtitles.dev', 'gutenberg.dev', 'childes.dev', 'bnc_spoken.dev']
the file:switchboard.dev has been appeneded to the uber list and its length is 1 
the file:simple_wiki.dev has been appeneded to the uber list and its length is 2 
the file:open_subtitles.dev has been appeneded to the uber list and its length is 3 
the file:gutenberg.dev has been appeneded to the uber list and its length is 4 
the file:childes.dev has been appeneded to the uber list and its length is 5 
the file:bnc_spoken.dev has been appeneded to the uber list and its length is 6 


### we read the tokenize data and store the input_ids and attention mask as a single long list at run time we reshape the single [1,B*T] tensor into a batched tensor of dimension [B,T].
This approach turns out to be more efficient as it removes the need for any extra tokens in the form of padding

In [12]:
def get_df_from_list(enc , B= B, T = T, val= False):
    """This function takes the tokenized list and reated a dataframe where each row contains the input_ids and attention_mask from the tokenizer.
    each row of the dataframe is a list with items B*T"""
    chunk_size = B*T
    if val:
        chunk_size = 2*B*T
        
    long_list_inp = enc['input_ids']
    long_list_attention = enc['attention_mask']

    # Step 3: Split the list into chunks and pad the last chunk if necessary
    chunks_inp = [long_list_inp[i:i + chunk_size] for i in range(0, len(long_list_inp), chunk_size)]
    chunks_att = [long_list_attention[i:i + chunk_size] for i in range(0, len(long_list_attention), chunk_size)]
    df = pd.DataFrame({'input_ids': chunks_inp,'attention_mask':chunks_att})
    return df
    

In [13]:
df_train_temp = get_df_from_list(enc_train)
# Display the DataFrame
df_train_temp.head()
print(f"Length of the dataframe is = {len(df_train_temp)}")

Length of the dataframe is = 525


In [14]:
df_val_temp = get_df_from_list(enc_val)
# Display the DataFrame
df_val_temp.head()
print(f"Length of the VALDATION dataframe is = {len(df_val_temp)}")


Length of the VALDATION dataframe is = 532


In [15]:
def pad_df(df,eos_char = tokenizer.eos_token_id,B = B, T = T,val = False):
    """The 2 functions below makes sure that each row of the dataframe is of equal length and if not it adds the eos token to make it B*T"""
    if val:
        B = 2*B
    for ind,row in df.iterrows():
        if len(row['input_ids']) != B*T :
            print(f"row = {ind} and input_id length = {len(row['input_ids'])}")
            print(f"row = {ind} and attention length = {len(row['attention_mask'])}")
            pad_len = B*T - len(row['input_ids'])
            print(f"padding the row index {ind} with {pad_len} character")
            row['input_ids'] = row['input_ids']+ [eos_char] * pad_len
            #attention mask should be padded to 0
            row['attention_mask'] = row['attention_mask']+ [0] * pad_len
            print("#### POST CONCAT####")
            print(f"row = {ind} and input_id length = {len(row['input_ids'])}")
            print(f"row = {ind} and attention length ={len(row['attention_mask'])}")
    return df


def verify_len(df,val = False, B= B):
    row_ind = []
    if val:
        B = 2*B
    for ind,row in df.iterrows():
        if len(row['input_ids']) != B*T :
            row_ind.append(ind)
        else:
            continue
    if len(row_ind) !=0:
        print("CONCATENATION Did not work")
    else:
        print("CONCATENATION worked")
        
            
        
    
        
        
   
   

In [16]:
df_train  = pad_df(df_train_temp)
verify_len(df_train)

row = 524 and input_id length = 21539
row = 524 and attention length = 21539
padding the row index 524 with 11229 character
#### POST CONCAT####
row = 524 and input_id length = 32768
row = 524 and attention length =32768
CONCATENATION worked


In [17]:
df_val  = pad_df(df_val_temp)
verify_len(df_val)

row = 531 and input_id length = 13588
row = 531 and attention length = 13588
padding the row index 531 with 19180 character
#### POST CONCAT####
row = 531 and input_id length = 32768
row = 531 and attention length =32768
CONCATENATION worked


In [18]:
df_train.head()

,input_ids,attention_mask
0,"[32, 25, 197, 40, 1101, 1654, 484, 389, 13, 19...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"[6510, 11, 14104, 290, 27913, 13, 198, 32, 25,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"[314, 1612, 11, 340, 338, 1611, 286, 43244, 11...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[318, 407, 922, 329, 262, 1200, 2035, 13, 198,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,"[198, 32, 25, 197, 1870, 11, 21480, 11, 314, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Define the model

In [19]:
class CustomLoss(nn.Module):
    def __init__(self, num_epochs):
        super(CustomLoss, self).__init__()
        #self.nll_loss = nn.CrossEntropyLoss()
#        self.cosine_similarity = nn.CosineSimilarity(dim=2)
        self.num_epochs = num_epochs
        self.current_epoch = 0
        self.nll_scale = 1.0  # Initialize with a default value

    def forward(self, nll,cosine_sim):
               
        nll_normalized = nll / self.nll_scale         
        # Calculate lambda value
        lambda_val = self.get_lambda()
        # Compute total loss
        total_loss = nll_normalized + lambda_val * cosine_sim
        #print(f"inside CustomLoss->nll = {nll}|nll_scale = {self.nll_scale}|nll_normalized= {nll_normalized}|cosine_sim = {cosine_sim} | lambda val = {lambda_val}|total_loss = {total_loss}")
        return total_loss,lambda_val,self.nll_scale

    def get_lambda(self):
        # Exponential increase from 0 to 1
        return 1 - math.exp(-5 * self.current_epoch / self.num_epochs)

    def update_epoch(self, epoch):
        self.current_epoch = epoch

    def update_nll_scale(self, nll_value):
        self.nll_scale = max(self.nll_scale, nll_value)
    def update_epoch(self, epoch):
        self.current_epoch = epoch
    def get_nll_scale(self):
        #print(self.nll_scale)
        return self.nll_scale

In [20]:
# Test the tokenizer:
model_name = "gpt2"
if random_init_wts:
    config = AutoConfig.from_pretrained(model_name, vocab_size = 50304)
    # Initialize the model with random weights
    model = AutoModelForCausalLM.from_config(config)
else:
    #model = AutoModelForCausalLM.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained("gpt2")

#model = torch.compile(model)


In [21]:
len(df_train)

525

In [22]:
# device = "cuda"
# inp = torch.tensor(df_train.iloc[5]['input_ids']).view(B,T).to(device)
# att = torch.tensor(df_train.iloc[5]['attention_mask']).view(B,T).to(device)
# lab = inp.clone()
# # lab = F.pad(lab[:, 1:], (0, 1), value=-100)  # Shift labels to the left and pad with -100
# # lab[:, -1] = -100
# model.to(device)
# model_out = model(input_ids = inp, attention_mask = att , labels = lab)
# assert not torch.isnan(inp).any(), "Input contains NaN"
# assert not torch.isinf(inp).any(), "Input contains inf"
# model_out

In [23]:
# print(f"Input shape: {inp.shape}")
# print(f"Attention mask shape: {att.shape}")
# print(f"Labels shape: {lab.shape}")

### Data loaders and Dataset for batched training

In [24]:
class dataset_pyt_val(Dataset):
    """Dataset for validation data"""
    def __init__(self, df, B = B, T = T ):
        self.df = df
        print(f"Value of B {B}")
                                        
    def __getitem__(self, idx):
        #print(f"inside loader...idx ->{idx}")
        input_id_temp = torch.tensor(self.df.iloc[idx]['input_ids'],dtype = torch.long)
        att_mask = torch.tensor(self.df.iloc[idx]['attention_mask'],dtype = torch.long)
        input_id =   input_id_temp.view(B,T)    
        attention_mask = att_mask.view(B,T)
        
        
        return input_id, attention_mask
        
    def __len__(self):
        #return the length of the dataframe
        return len(self.df)

In [25]:
class dataset_pyt_train(Dataset):
    def __init__(self, df, B = B, T = T ):
        self.df = df
                                        
    def __getitem__(self, idx):
        #print(f"************* idx for dataloader = {idx}")
        input_id_temp = torch.tensor(self.df.iloc[idx]['input_ids'],dtype = torch.long)
        att_mask = torch.tensor(self.df.iloc[idx]['attention_mask'],dtype = torch.long)
        input_id =   input_id_temp.view(B,T)    
        attention_mask = att_mask.view(B,T)   
        
        return input_id, attention_mask
        
    def __len__(self):
        #return the length of the dataframe
        return len(self.df)

### Note - batch_size is 1 here because we reshape our input data in the shape [B,T] for a batched training.

In [26]:
#train_dataset = dataset_pyt(filtered_df,tokenizer = tokenizer)
train_dataset = dataset_pyt_train(df_train)
val_dataset = dataset_pyt_val(df_val)
#test_dataset = dataset_pyt(df_test,tokenizer = tokenizer)

train_loader = DataLoader(train_dataset,batch_size = 1, shuffle = True , num_workers = 0, pin_memory = True)
val_loader = DataLoader(val_dataset,batch_size = 1, shuffle = True , num_workers = 0, pin_memory = True)
#test_loader = DataLoader(test_dataset,batch_size = batch_size, shuffle = False, collate_fn = custom_collate_fn)


Value of B 32


In [27]:
print(f"Length of the train loader is {len(train_loader)}")
print(f"Length of the val loader is {len(val_loader)}")
print(f"num_tokens= {B*T*len(train_loader)}")

Length of the train loader is 525
Length of the val loader is 532
num_tokens= 17203200


In [28]:
def write_file(log_message, model_name = model_name ,random_init_wts = random_init_wts ):
    """This function logs the training performance for future reference"""
    current_datetime = datetime.datetime.now()
    # Extract date and time components
    current_date = str(current_datetime.date())
    log_file = model_name +'_COS_SIM*lambda_'+'random_init_wts'+ '_'+str(random_init_wts)+'_' +current_date+'.log'
    print(f"*****LOGGING INFO IN {log_file}*********")
    filepath = os.path.join("model",log_file)
    logging.basicConfig(filename=filepath, 
                    filemode='a',  # Overwrite the log file each time
                    format='%(asctime)s - %(levelname)s - %(message)s', 
                    level=logging.DEBUG)
    logger = logging.getLogger()
    logger.info(log_message)
    
    

    
    

In [29]:
@torch.no_grad

def eval_model(val_loader, model, lambda_val ,nll_scale,epoch , device = device,tokenizer = tokenizer):
    global global_val_loss
    model.eval()
    model.to(device)
    e = epoch+1
    embedding_layer = model.transformer.wte
    val_loss_accum = 0.0
    print(f"inside validation data for epoch {e}")
    for ind,(input_id,attention_mask) in enumerate(val_loader):
        ids = input_id.to(device=device, non_blocking=True)
        att_mask = attention_mask.to(device=device, non_blocking=True)
        labels = ids.clone()
        
        with autocast(dtype = torch.bfloat16):
            input_embeddings = embedding_layer(ids)
            model_output = model(input_ids = ids ,attention_mask = att_mask, labels = labels)
            logits = model_output.logits
            predictions = torch.argmax(logits, dim=-1)
            #print(f"predictions = {predictions}")
            prediction_embeddings = embedding_layer(predictions)
            cos_sim = F.cosine_similarity(torch.squeeze(prediction_embeddings,dim = 0), torch.squeeze(input_embeddings,dim = 0), dim=1)
            cos_loss = 1- cos_sim.mean()
            total_loss = cos_loss + model_output.loss
        
        val_loss_accum+=total_loss.detach().item()
        del ids,att_mask,labels,input_embeddings,model_output,logits,predictions,prediction_embeddings,cos_sim,cos_loss
    
    #logging and saving if the validation loss has decreased.
    if val_loss_accum < global_val_loss:
        print(f"Val loss has decreased -->reducing the global validation loss from {global_val_loss:.2f} to {val_loss_accum:.2f}")
        s1 = f"Val loss has decreased -->reducing the global validation loss from {global_val_loss:.2f} to {val_loss_accum:.2f}"
        global_val_loss = val_loss_accum
        print(f" validation loss for epoch = {e} is {val_loss_accum:.4f}")
        s2 = f" validation loss for epoch = {e} is {val_loss_accum:.4f}"
        print(f" epoch= {e} :  val loss is {val_loss_accum:.4f} ")
        s3 = f" epoch= {e} :  val loss is {val_loss_accum:.4f} "
        #save the model
        
        # Get the current date and time
        current_datetime = datetime.datetime.now()
        # Extract date and time components
        current_date = str(current_datetime.date())
        current_time = str(current_datetime.time()).split('.')[0]
        file_name = 'model'+ current_date+current_time+'.pth'
        path = os.path.join("model",file_name)
        print(f"saving the model {file_name}")
        s4 = f"saving the model {file_name}"
        #torch.save(model.state_dict(), path)
        model.save_pretrained(path)
        tokenizer.save_pretrained(path)
        log_message = s1+s2+s3+s4
        write_file(log_message)
        
        #plot_confusion_matrix(y_val.cpu().numpy(), y_hat_val.cpu().numpy(), labels)
    else:
        print(f"No improvement in validation loss-->epoch= {e} and global val loss is {global_val_loss:.2f}|current_Val loss = {val_loss_accum}")
    return val_loss_accum
        
    
    
    


In [30]:
def train_model(train_loader,val_loader,model,num_epoch = 100,device = device,tokenizer = tokenizer):
    global global_tr_loss
    model.train()
    device = device
    lr_custom = 2e-6
    print(f"inside train model. Device = {device}")
    model.to(device)
    optimizer = torch.optim.AdamW(params =  model.parameters(), lr= lr_custom,fused = True ,weight_decay = .1)
    total_batch_size = 2**19
    grad_accum_step = total_batch_size//(B*T)
    
    extra_train = .1*num_epoch
    max_train_steps = int(num_epoch +extra_train )
    import time
    from transformers import get_linear_schedule_with_warmup
    total_steps = len(train_loader) * num_epoch
    scheduler_cos = transformers.get_cosine_schedule_with_warmup( optimizer= optimizer, num_warmup_steps =int(total_steps * 0.1) ,num_training_steps= total_steps )
    embedding_layer = model.transformer.wte
    criterion = CustomLoss(num_epoch)
    epoch_train_log = []
    epoch_val_log = []
    for i in range (max_train_steps):
        criterion.update_epoch(i)
        epoch_start_time = time.time()
        epoch_train_loss = 0.0
        optimizer.zero_grad(set_to_none=True)
        # we use 2 schedulers - the first LR scheduler uses a cosine decay for 100 epochs the second scheduler takes the last LR from cosine scheduler and then maintains that LR for the next 10 epochs
        if i >= num_epoch:
            optimizer_reduced_lr = torch.optim.AdamW(params =  model.parameters(), lr= current_lr ,fused = True , weight_decay=.1)
            scheduler_constant = transformers.get_constant_schedule_with_warmup( optimizer = optimizer_reduced_lr ,num_warmup_steps = 0, last_epoch = -1 )
        
        lambda_val = None
        nll_scale = None
        epoch_nll = []
        
        for ind,(input_id,attention_mask) in enumerate(train_loader):
            if ind == int(len(train_loader)/2):
                batch_time = time.time()
                duration = batch_time - epoch_start_time
                print(f"executing epoch:{i+1}, it took {duration/60} mins from beginning of epoch till batch#{ind}")
            
            ids = input_id.to(device=device, non_blocking=True)
            att_mask = attention_mask.to(device=device, non_blocking=True)
            labels = ids.clone()
            assert not torch.isnan(ids).any(), "Input contains NaN values"
            assert not torch.isinf(ids).any(), "Input contains infinite values"
            
            with autocast(dtype = torch.bfloat16):
                input_embeddings = embedding_layer(ids)
                model_output = model(input_ids = ids ,attention_mask = att_mask, labels = labels)
                logits = model_output.logits
                predictions = torch.argmax(logits, dim=-1)
                prediction_embeddings = embedding_layer(predictions)
                #print(f"model_output.logits = {model_output.logits}|model_output.loss = {model_output.loss}")
                cos_sim = F.cosine_similarity(torch.squeeze(prediction_embeddings,dim = 0), torch.squeeze(input_embeddings,dim = 0), dim=1)
                cos_loss = 1- cos_sim.mean()
                #print(f"inside train_vale before calc custom loss|ind = {ind}|model_output.loss = {model_output.loss}|coss_loss = {cos_loss}")
                #print(f"inside train_vale before calc custom loss|ind = {ind}|model_output.loss = {model_output.loss}|coss_loss = {cos_loss}|learning rate = {scheduler_cos.get_last_lr()[0]:.3e}")
                if np.isnan(model_output.loss.item()):
                    print("f nan values encountered..")
                    decoded_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in ids]
                    print(f"*********$$$$$$$$$ decoded_texts = {decoded_texts}*************")
                total_loss,lambda_val,nll_scale = criterion(nll = model_output.loss.detach().item() , cosine_sim = cos_loss)
                #print(f"Current value of nll max is {epoch_nll_max}| current mini batch loss is = {model_output.loss.item()}")
                epoch_nll.append(model_output.loss.detach())
                #print(f"batch index = {ind}|epoch_nll_max = {epoch_nll_max}")


            assert not np.isnan(model_output.loss.item()), "NaN value found"
            #total_loss = cos_loss + model_output.loss.item()
            
            if ind == (len(train_loader) - 1):
                lambda_val = lambda_val
                nll_scale = nll_scale
                
            
            total_loss.backward()
            epoch_train_loss += total_loss.detach().item()
            norm = torch.nn.utils.clip_grad_norm(model.parameters() , 1.0)
            if i <= num_epoch:
                optimizer.step()
                scheduler_cos.step()
                optimizer.zero_grad(set_to_none=True)
            else:
                optimizer_reduced_lr.step()
                optimizer_reduced_lr.zero_grad(set_to_none=True)
                scheduler_constant.step()
                
                                
            del ids,att_mask,labels,input_embeddings,model_output,logits,predictions,prediction_embeddings,cos_sim
            
        
        
        #batch processing complete 
        print(f"batch processing complete , before updating nll_scale,updating the nll_scale from {criterion.get_nll_scale()} to {torch.mean(torch.tensor(epoch_nll))}|total_loss for batch= {total_loss}")
        mean_nll = torch.mean(torch.tensor(epoch_nll))
        criterion.update_nll_scale(torch.mean(mean_nll).item())                           
        #print(f"batch processing complete , after updating nll_scale , value is {epoch_nll_max}")
        if i <= num_epoch:
            current_lr = scheduler_cos.get_last_lr()[0]
        #This code below compares the training loss during the current epoch with the global loss and if updates the global loss if there is an improvement.
        # at every 4 epoch we evluate the model on the validation data or when 
        epoch_end_time = time.time()
        epoch_durn = (epoch_end_time - epoch_start_time)
        num_token = B*T*len(train_loader)
        epoch_train_log.append(epoch_train_loss)
        if (epoch_train_loss < global_tr_loss) :
            improvement_percent = (abs(epoch_train_loss-global_tr_loss)/global_tr_loss)*100
            print(f"training loss has decreased---> reducing the global loss from {global_tr_loss:.2f} to {epoch_train_loss:.2f} | throughput = {int(num_token/epoch_durn)} tokens/second | norm = {norm:.4f} | learning rate = {current_lr:.5e}")
            global_tr_loss = epoch_train_loss
            print(f" epoch= {i+1} and  train loss is {epoch_train_loss:.2f}")
            if (i%4 == 0 or improvement_percent > 5):
                val_loss= eval_model(val_loader, model, lambda_val ,nll_scale,epoch = i, device = device,tokenizer = tokenizer)
                epoch_val_log.append(val_loss)
            
        else:
            print(f"No improvement in training loss..the global training loss is -->{global_tr_loss:.2f}|epoch train_loss = {epoch_train_loss:.2f} ")
            #print(f" epoch= {i+1} and mean train loss is {epoch_train_loss:.2f}")
        
        
    
    return model,epoch_train_log,epoch_val_log
        
            
            
    
    

In [31]:
tr_model,epoch_train_log,epoch_val_log = train_model(train_loader, val_loader, model =  model,tokenizer = tokenizer)

inside train model. Device = cuda:0
executing epoch:1, it took 0.8258710503578186 mins from beginning of epoch till batch#262
batch processing complete , before updating nll_scale,updating the nll_scale from 1.0 to 3.4965267181396484|total_loss for batch= 3.8043603897094727
training loss has decreased---> reducing the global loss from inf to 1835.68 | throughput = 173516 tokens/second | norm = 0.0000 | learning rate = 2.00000e-07
 epoch= 1 and  train loss is 1835.68
inside validation data for epoch 1
Val loss has decreased -->reducing the global validation loss from inf to 2253.45
 validation loss for epoch = 1 is 2253.4457
 epoch= 1 :  val loss is 2253.4457 
saving the model model2024-07-1601:22:33.pth
*****LOGGING INFO IN gpt2_COS_SIM*lambda_random_init_wts_False_2024-07-16.log*********
executing epoch:2, it took 0.8133656501770019 mins from beginning of epoch till batch#262
batch processing complete , before updating nll_scale,updating the nll_scale from 3.4965267181396484 to 3.3506

In [32]:
import json
# Write the list to a JSON file
with open("epoch_train_log_mean_nll.json", "w") as file:
    json.dump(epoch_train_log, file)

with open("epoch_val_log_mean_nll.json", "w") as file:
    json.dump(epoch_val_log, file)

